数据读取

In [1]:
import pandas as pd


# 1. 数据导入，在pd模块中有csv的处理函数
train_data = pd.read_csv("E:\\xunlei\\train\\train.csv")
test_data = pd.read_csv("E:\\xunlei\\train\\test.csv")

2. 数据预处理，我觉得summary肯定是没用的，address我也不能理解，因为每个房子的address一定是不同的，第二个就是就是state 都是ca
根据网上的信息，pandas删除列的Api是del，传入的是字符串，代表的是列的名字
所以第一步，将需要删除的列安在一个数组，遍历这个数组进行删除


知识预备：pandas对数值型数据和非数值型数据的处理，
pd.read_csv读入时，如果读入的是字符串，那么该列会以object形式读入，也不能直接进行计算，所以需要对数据进行预处理，清除掉非数字的数据

In [2]:
delet_list = ['Address','Summary','State']
for index in delet_list:
    del train_data[index],test_data[index]
train_data

,Id,Sold Price,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,Bathrooms,...,Laundry features,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip
0,0,3825000.0,SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,"Ground Floor Bedroom, Master Bedroom on Ground...",0.0,...,"Washer / Dryer, Inside, In Utility Room","Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022
1,1,505000.0,SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,3,2.0,...,Inside,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047
2,2,140000.0,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,2,3.0,...,NaN,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375
3,3,1775000.0,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",NaN,5,3.0,...,"Stackable, Upper Level","Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230
4,4,1175000.0,VacantLand,NaN,NaN,NaN,0 spaces,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47434,47434,159000.0,SingleFamily,1965.0,Central,Central Air,"Carport, Covered",20908.8,3,2.0,...,NaN,"Carport, Covered",125049.0,1298.0,2020-09-04,155000.0,NaN,NaN,Blythe,92225
47435,47435,255000.0,MobileManufactured,1999.0,"Central Forced Air, Gas",Central AC,Carport,NaN,3,2.0,...,Hookups Only,Carport,68924.0,721.0,2020-07-23,295900.0,NaN,NaN,Sunnyvale,94089
47436,47436,2300000.0,SingleFamily,1919.0,Central,Central Air,Driveway,6756.0,3,2.0,...,Laundry Room,Driveway,578000.0,7211.0,2020-12-01,2295000.0,2015-05-01,2150000.0,Los Angeles,90046
47437,47437,500000.0,SingleFamily,2017.0,Central,Central Air,"Garage, Carport, Garage - Attached, Covered",5945.0,3,3.0,...,Inside Room,"Garage, Carport, Garage - Attached, Covered",484696.0,7627.0,2020-08-01,499888.0,2020-08-24,500000.0,Elk Grove,95757


删除id和没用的东西
本来想像老师一样，进行同批次处理的，但是发现执行的过程中有错误，我个人的排查，可能是因为all_features = pd.concat((train_data.iloc[:, 1:], test_data.iloc[:, 1:]),axis=1)

In [3]:
# all_features = pd.concat((train_data.iloc[:, 1:], test_data.iloc[:, 1:]),axis=1)
def resizeNumerica(train_data):
    numeric_feature = train_data.dtypes[train_data.dtypes != 'object'].index
    train_data[numeric_feature] = train_data[numeric_feature].apply(lambda x: (x - x.mean()) / (x.std()))
    train_data[numeric_feature] = train_data[numeric_feature].fillna(0)
    return train_data,numeric_feature
train_data,train_numeric_index = resizeNumerica(train_data.iloc[:, 1:])
test_data,test_numeric_index = resizeNumerica(test_data.iloc[:,1:])

接下来就是对字符串数据进行独热编码
在进行独热编码之前，可以对特殊值进行分析，到底有多少种类，我如果一下自动生成 内存就爆了， 而且还变成了4W个column
我们没有对语义进行分析，只是单纯的进行类别分析，所以大类型的可以直接丢弃,根据下面的输出，我只保留Type,Bedrooms,Cooling features 其他的不再进行编码
思路就是对Numerical进行扩展，只保留需要保存的列

In [4]:
# train_data = pd.get_dummies(train_data)
# test_data = pd.get_dummies(test_data)
for in_object in train_data.dtypes[train_data.dtypes=='object'].index:
    print(in_object.ljust(20),len(train_data[in_object].unique()))

Type                 158
Heating              1861
Cooling              542
Parking              5114
Bedrooms             265
Region               950
Elementary School    1718
Middle School        489
High School          631
Flooring             1349
Heating features     1123
Cooling features     313
Appliances included  4585
Laundry features     1977
Parking features     4961
Listed On            2467
Last Sold On         6114
City                 929
